# Improt libraries

In [225]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import TensorBoard
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

# Load dataset

In [226]:
path = "gesture_data"

data = []
labels = []

for file_name in os.listdir(path):
    if file_name .endswith(".csv"):
        label = file_name.split("_")[0]
        file_path = os.path.join(path, file_name)

        df = pd.read_csv(file_path)
        data.append(df.values)
        labels.append(label)

In [227]:
np.array(data).shape, np.array(labels).shape

((46, 30, 1629), (46,))

# Split dataset

In [228]:
X = np.array(data)
y = np.array(labels)

In [229]:
label_encoder = LabelEncoder()
y_encoded = to_categorical(label_encoder.fit_transform(labels)) 

# Build Model

In [230]:
model = keras.Sequential([
    Input(shape=(30, 1629)),
    LSTM(64, return_sequences=True, activation='relu'),
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') # Change the number of output as needed
])

In [231]:
adam = Adam(learning_rate=0.0001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model

In [232]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,)
history = model.fit(
    X,y_encoded,
    epochs=100,
    validation_split=0.2,
    batch_size=16,
    callbacks=callback
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.2405 - loss: 1.3847 - val_accuracy: 0.4000 - val_loss: 1.3719
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2795 - loss: 1.3776 - val_accuracy: 0.4000 - val_loss: 1.3657
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3186 - loss: 1.3722 - val_accuracy: 0.4000 - val_loss: 1.3540
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2951 - loss: 1.3583 - val_accuracy: 0.4000 - val_loss: 1.3411
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2561 - loss: 1.3555 - val_accuracy: 0.4000 - val_loss: 1.3281
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2717 - loss: 1.3419 - val_accuracy: 0.4000 - val_loss: 1.2889
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2561 - loss: 1.3107 - val_accuracy: 0.4000 - val_loss: 1.2362
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2795 - loss: 1.2571 - val_accuracy: 0.4000 - val_loss: 1

# Save model

In [233]:
model.save('gesture_classifier.h5')

# Save class mapping

In [234]:
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class Mapping:")
for class_name, encoded_value in class_mapping.items():
    print(f"{class_name}: {encoded_value}")

Class Mapping:
call: 0
hi: 1
same: 2
thanks: 3


In [235]:
file_name = "class_mapping.txt"
with open(file_name, "w") as file:
    file.write("Class Mapping:\n")
    for class_name, encoded_value in class_mapping.items():
        file.write(f"{class_name}: {encoded_value}\n")

print(f"Class mapping saved to {file_name}")

Class mapping saved to class_mapping.txt
